In [198]:
import pandas as pd
# import csv_utils
import string

from datetime import datetime
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

In [199]:
RAW_DATA = './../bbc_data_raw.csv'

In [200]:
df = pd.read_csv(RAW_DATA, sep=';')
# df = pd.read_csv('./../data/BBC/bbc_images.csv', sep=';', index_col=0)

In [201]:
df

,id,title,subtitle,description,image,keywords,synopses_small,synops_med,synops_long,category,channel,language,release_date,duration_sec,topic
0,0,The 80s - Music’s Greatest Decade?,Series 1: 1. With Dylan Jones,Music author Dylan Jones argues that the 1980s...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, The 80s - Music’s Greatest D...",Music author Dylan Jones argues that the 1980s...,Music writer Dylan Jones puts the case that th...,Eighties music is often dismissed as a joke - ...,Music,bbc_two,False,8:55pm 23 Oct 2021,3534.0,music
1,1,Top of the Pops,02/07/1992,"With Arrested Development, Altern 8, Def Leppa...",https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Top of the Pops, 02/07/1992","With Arrested Development, Altern 8, Def Leppa...","Tony Dortie presents the pop chart programme, ...",NaN,Music,bbc_one,False,7pm 2 Jul 1992,1759.0,music
2,2,BBC Proms,"2021: Last Night of the Proms, Part 2",Australian tenor Stuart Skelton is joined by L...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, BBC Proms, 2021: Last Night ...",Australian tenor Stuart Skelton is joined by L...,Australian tenor Stuart Skelton is joined by L...,Katie Derham hosts continued live coverage fro...,Music,bbc_radio_three,False,9pm 11 Sep 2021,5247.0,music
3,3,Port,Series 5: Episode 4,"Previously unseen music from the series Port, ...",https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Port, Series 5: Episode 4","Previously unseen music from the series Port, ...",Julie Fowlis a’ lìbhrigeadh ceòl bhon t-sreath...,Bidh Druthag Bheag Eile a’ tarraing ri chèile ...,Music,bbc_alba,False,9 Jul 2020,1037.0,music
4,4,"Sit Down, Stand Up with Greg James",NaN,Tom Holland and Greg James cold call celebrity...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Sit Down, Stand Up with Greg...",Tom Holland and Greg James cold call celebrity...,Tom Holland and Greg James cold call celebrity...,NaN,Entertainment,bbc_radio_one,False,10 Feb 2022,611.0,music
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36893,36896,Justin's House,Series 2: 4. Going for Gold,"Justin's house hosts the Justinship Games, and...",https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Justins House, Series 2: 4. ...","Justin's house hosts the Justinship Games, and...",Justin Fletcher entertains children with singi...,Justin Fletcher entertains a live audience of ...,CBeebies,cbeebies,False,11am 2 Sep 2012,1454.0,cbeebees
36894,36897,Number 1 Newton Avenue,Series 1: 21. Goal,It’s all about measuring today when Libby play...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Number 1 Newton Avenue, Seri...",It’s all about measuring today when Libby play...,Libby plays a game of football with her teddie...,Libby plays a game of football with her teddie...,CBeebies,cbeebies,False,4:55pm 20 Sep 2021,431.0,cbeebees
36895,36898,Biggleton,Series 2: 23. Big Day Out,Odd Job Laila has lots of jobs to do around to...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Biggleton, Series 2: 23. Big...",Odd Job Laila has lots of jobs to do around to...,Odd Job Laila has lots of jobs to do around to...,The Big Day Out has finally arrived in Bigglet...,CBeebies,cbeebies,False,14 Jan 2019,651.0,cbeebees
36896,36899,Go Jetters,"Series 1: 27. The Strokkur Geyser, Iceland",The geysers in Iceland are blocked and Glitch ...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Go Jetters, Series 1: 27. Th...",The geysers in Iceland are blocked and Glitch ...,Animated adventure comedy series. The geysers ...,NaN,CBeebies,cbeebies,False,5:25pm 16 Jun 2016,659.0,cbeebees


In [202]:
columns = ['index', 'title', 'subtitle', 'description', 'image', 'keywords', 'syno_s', 'syno_m', 'syno_l', 'category', 'channel', 'language', 'release_date', 'duration_sec', 'topic']
#
df.columns = columns

In [203]:
df

,index,title,subtitle,description,image,keywords,syno_s,syno_m,syno_l,category,channel,language,release_date,duration_sec,topic
0,0,The 80s - Music’s Greatest Decade?,Series 1: 1. With Dylan Jones,Music author Dylan Jones argues that the 1980s...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, The 80s - Music’s Greatest D...",Music author Dylan Jones argues that the 1980s...,Music writer Dylan Jones puts the case that th...,Eighties music is often dismissed as a joke - ...,Music,bbc_two,False,8:55pm 23 Oct 2021,3534.0,music
1,1,Top of the Pops,02/07/1992,"With Arrested Development, Altern 8, Def Leppa...",https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Top of the Pops, 02/07/1992","With Arrested Development, Altern 8, Def Leppa...","Tony Dortie presents the pop chart programme, ...",NaN,Music,bbc_one,False,7pm 2 Jul 1992,1759.0,music
2,2,BBC Proms,"2021: Last Night of the Proms, Part 2",Australian tenor Stuart Skelton is joined by L...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, BBC Proms, 2021: Last Night ...",Australian tenor Stuart Skelton is joined by L...,Australian tenor Stuart Skelton is joined by L...,Katie Derham hosts continued live coverage fro...,Music,bbc_radio_three,False,9pm 11 Sep 2021,5247.0,music
3,3,Port,Series 5: Episode 4,"Previously unseen music from the series Port, ...",https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Port, Series 5: Episode 4","Previously unseen music from the series Port, ...",Julie Fowlis a’ lìbhrigeadh ceòl bhon t-sreath...,Bidh Druthag Bheag Eile a’ tarraing ri chèile ...,Music,bbc_alba,False,9 Jul 2020,1037.0,music
4,4,"Sit Down, Stand Up with Greg James",NaN,Tom Holland and Greg James cold call celebrity...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Sit Down, Stand Up with Greg...",Tom Holland and Greg James cold call celebrity...,Tom Holland and Greg James cold call celebrity...,NaN,Entertainment,bbc_radio_one,False,10 Feb 2022,611.0,music
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36893,36896,Justin's House,Series 2: 4. Going for Gold,"Justin's house hosts the Justinship Games, and...",https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Justins House, Series 2: 4. ...","Justin's house hosts the Justinship Games, and...",Justin Fletcher entertains children with singi...,Justin Fletcher entertains a live audience of ...,CBeebies,cbeebies,False,11am 2 Sep 2012,1454.0,cbeebees
36894,36897,Number 1 Newton Avenue,Series 1: 21. Goal,It’s all about measuring today when Libby play...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Number 1 Newton Avenue, Seri...",It’s all about measuring today when Libby play...,Libby plays a game of football with her teddie...,Libby plays a game of football with her teddie...,CBeebies,cbeebies,False,4:55pm 20 Sep 2021,431.0,cbeebees
36895,36898,Biggleton,Series 2: 23. Big Day Out,Odd Job Laila has lots of jobs to do around to...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Biggleton, Series 2: 23. Big...",Odd Job Laila has lots of jobs to do around to...,Odd Job Laila has lots of jobs to do around to...,The Big Day Out has finally arrived in Bigglet...,CBeebies,cbeebies,False,14 Jan 2019,651.0,cbeebees
36896,36899,Go Jetters,"Series 1: 27. The Strokkur Geyser, Iceland",The geysers in Iceland are blocked and Glitch ...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Go Jetters, Series 1: 27. Th...",The geysers in Iceland are blocked and Glitch ...,Animated adventure comedy series. The geysers ...,NaN,CBeebies,cbeebies,False,5:25pm 16 Jun 2016,659.0,cbeebees


## Missing Data processing

In [204]:
df.isna().sum()

index              0
title            450
subtitle        1376
description      140
image            450
keywords         450
syno_s           450
syno_m          1255
syno_l          7450
category        1119
channel          450
language         450
release_date     452
duration_sec     450
topic              0
dtype: int64

In [205]:
df_nona = df.dropna(subset=['image', 'release_date', 'description'])

In [206]:
df_nona.isna().sum()

index              0
title              0
subtitle        1352
description        0
image              0
keywords           0
syno_s             0
syno_m           805
syno_l          7000
category         669
channel            0
language           0
release_date       0
duration_sec       0
topic              0
dtype: int64

In [207]:
df_nona.loc[df_nona['category'].isna(), 'category'] = df_nona[df_nona['category'].isna()]['topic']

In [208]:
df_nona.isna().sum()

index              0
title              0
subtitle        1352
description        0
image              0
keywords           0
syno_s             0
syno_m           805
syno_l          7000
category           0
channel            0
language           0
release_date       0
duration_sec       0
topic              0
dtype: int64

In [209]:
df = df_nona

## Image Processing

In [210]:
s = '192x108'
m = '288x162'
l = '352x198'
xl = '1200x675'
def generate_img(df_inp, size):
    return [image.replace('{recipe}', size) for image in df_inp['image']]
df_nona['image_l'] = generate_img(df_nona, l)
df_nona['image_xl'] = generate_img(df_nona, xl)

/var/folders/2s/d47mm4m157l7p7yy3xrkwqsh0000gn/T/ipykernel_22040/3820794576.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nona['image_l'] = generate_img(df_nona, l)
/var/folders/2s/d47mm4m157l7p7yy3xrkwqsh0000gn/T/ipykernel_22040/3820794576.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nona['image_xl'] = generate_img(df_nona, xl)


In [211]:
# df_img = df.loc[:, ['index', 'image', 'image_s', 'image_m', 'image_l']]
# df_img.to_csv('../data/BBC/bbc_images.csv', sep=';')

## Synopses clearing

In [212]:
len(df[df['syno_s'].isna()])

0

In [213]:
len(df[df['syno_m'].isna()])

805

In [214]:
len(df[df['syno_l'].isna()])

7000

In [215]:
def fill_column(df_input, origin_col, gap_filler_col):
    for index, df_entry in df_input.iterrows():
        # checks if the value is the same, if not it is a NaN
        if not df_entry[origin_col] == df_entry[origin_col]:
            df_input.loc[index, [origin_col]] = df_entry[gap_filler_col]

In [216]:
# Substitute all nan syno_m with the syno_s values
fill_column(df, origin_col='syno_m', gap_filler_col='syno_s')
# Substitute all nan syno_l with the syno_m values
fill_column(df, origin_col='syno_l', gap_filler_col='syno_m')

In [217]:
# df.to_csv('../data/BBC/bbc_data.csv', sep=';')

In [218]:
df.loc[df['subtitle'].isna(), 'subtitle'] = 'None'

In [219]:
df.isna().sum()

index           0
title           0
subtitle        0
description     0
image           0
keywords        0
syno_s          0
syno_m          0
syno_l          0
category        0
channel         0
language        0
release_date    0
duration_sec    0
topic           0
image_l         0
image_xl        0
dtype: int64

In [220]:
df = df.iloc[:, 1:]

## Remove duplicates

In [221]:
df[df['title'] == 'A Life in Ten Pictures']

,title,subtitle,description,image,keywords,syno_s,syno_m,syno_l,category,channel,language,release_date,duration_sec,topic,image_l,image_xl
14141,A Life in Ten Pictures,Series 1: 6. Amy Winehouse,"Ten pictures, from iconic shots to private sna...",https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, A Life in Ten Pictures, Seri...","Ten pictures, from iconic shots to private sna...",Amy Winehouse’s image is known around the worl...,Amy Winehouse’s image is known around the worl...,History,bbc_two,Contains strong language.,9pm 11 Sep 2021,3550.0,signed,https://ichef.bbci.co.uk/images/ic/352x198/p09...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...
14490,A Life in Ten Pictures,Series 1: 2. Tupac Shakur,"Ten pictures, from iconic shots to private sna...",https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, A Life in Ten Pictures, Seri...","Ten pictures, from iconic shots to private sna...",Tupac Shakur’s image is famous around the worl...,Tupac Shakur’s image is known around the world...,History,bbc_two,Contains some strong language.,10 Apr 2021,3531.0,signed,https://ichef.bbci.co.uk/images/ic/352x198/p09...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...
14750,A Life in Ten Pictures,Series 1: 1. Freddie Mercury,"Ten pictures, from iconic shots to private sna...",https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, A Life in Ten Pictures, Seri...","Ten pictures, from iconic shots to private sna...",Freddie Mercury’s image is famous around the w...,Freddie Mercury’s image is known around the wo...,History,bbc_two,Contains some strong language.,9pm 3 Apr 2021,3532.0,signed,https://ichef.bbci.co.uk/images/ic/352x198/p09...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...
17236,A Life in Ten Pictures,Series 1: 5. Muhammad Ali,"Ten pictures, from iconic shots to private sna...",https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, A Life in Ten Pictures, Seri...","Ten pictures, from iconic shots to private sna...",Muhammad Ali is one of the most photographed m...,Muhammad Ali is one of the most photographed m...,History,bbc_two,False,9pm 8 May 2021,3546.0,signed,https://ichef.bbci.co.uk/images/ic/352x198/p09...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...
17420,A Life in Ten Pictures,Series 1: 3. Elizabeth Taylor,"Ten pictures, from iconic shots to private sna...",https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, A Life in Ten Pictures, Seri...","Ten pictures, from iconic shots to private sna...",Elizabeth Taylor’s image is famous around the ...,Elizabeth Taylor’s image is known around the w...,History,bbc_two,Contains some strong language.,9:10pm 17 Apr 2021,3556.0,signed,https://ichef.bbci.co.uk/images/ic/352x198/p09...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...
18568,A Life in Ten Pictures,Series 1: 4. John Lennon,"Ten pictures, from iconic shots to private sna...",https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, A Life in Ten Pictures, Seri...","Ten pictures, from iconic shots to private sna...",John Lennon’s image is famous around the world...,John Lennon’s image is known around the world;...,History,bbc_two,False,9pm 15 May 2021,3539.0,signed,https://ichef.bbci.co.uk/images/ic/352x198/p09...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...
25359,A Life in Ten Pictures,Series 1: 1. Freddie Mercury,"Ten pictures, from iconic shots to private sna...",https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, A Life in Ten Pictures, Seri...","Ten pictures, from iconic shots to private sna...",Freddie Mercury’s image is famous around the w...,Freddie Mercury’s image is known around the wo...,History,bbc_two,Contains some strong language.,9pm 3 Apr 2021,3532.0,history,https://ichef.bbci.co.uk/images/ic/352x198/p09...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...
25483,A Life in Ten Pictures,Series 1: 2. Tupac Shakur,"Ten pictures, from iconic shots to private sna...",https://ichef.bbci.co.uk/images/ic/{

In [222]:
df.groupby(['title', 'subtitle']).size()

title                                      subtitle                                 
#CancelKarenDunbar                         None                                         2
'Go Back To Where You Came From'           None                                         1
'Hideous and Barbaric Venture'             None                                         1
'Til Kingdom Come: Trump, Faith and Money  None                                         1
04 Wal                                     Cyfres 10: Pennod 10                         1
                                                                                       ..
’S E Iasg a Th’Annam (I’m a Fish)          Series 1: 47. Brod-iasg                      1
                                           Series 1: 48. Bradan-leathann Chalifornia    1
                                           Series 1: 5. Òr-iasg                         1
                                           Series 1: 50. Gibearnach                     1
               

In [223]:
grouped_by_title_subtitle = df.groupby(['title', 'subtitle'])
def remove_duplicates_by_title(df, grouped_by_title_subtitle):
    for group, entries in grouped_by_title_subtitle:
        if len(entries) == 1:
            continue

        indices = []
        topics = []

        for i, value in entries.iterrows():
            topics.append(value['topic'])
            indices.append(i)
        df.loc[indices[0], 'topic'] = ', '.join(topics)
        df = df.drop(labels=indices[1:], axis=0)
    return df
df = remove_duplicates_by_title(df, grouped_by_title_subtitle)

In [224]:
df.groupby(['title', 'subtitle']).size()

title                                      subtitle                                 
#CancelKarenDunbar                         None                                         1
'Go Back To Where You Came From'           None                                         1
'Hideous and Barbaric Venture'             None                                         1
'Til Kingdom Come: Trump, Faith and Money  None                                         1
04 Wal                                     Cyfres 10: Pennod 10                         1
                                                                                       ..
’S E Iasg a Th’Annam (I’m a Fish)          Series 1: 47. Brod-iasg                      1
                                           Series 1: 48. Bradan-leathann Chalifornia    1
                                           Series 1: 5. Òr-iasg                         1
                                           Series 1: 50. Gibearnach                     1
               

In [225]:
df[df['title'] == 'A Life in Ten Pictures']

,title,subtitle,description,image,keywords,syno_s,syno_m,syno_l,category,channel,language,release_date,duration_sec,topic,image_l,image_xl
14141,A Life in Ten Pictures,Series 1: 6. Amy Winehouse,"Ten pictures, from iconic shots to private sna...",https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, A Life in Ten Pictures, Seri...","Ten pictures, from iconic shots to private sna...",Amy Winehouse’s image is known around the worl...,Amy Winehouse’s image is known around the worl...,History,bbc_two,Contains strong language.,9pm 11 Sep 2021,3550.0,"signed, history, documentaries",https://ichef.bbci.co.uk/images/ic/352x198/p09...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...
14490,A Life in Ten Pictures,Series 1: 2. Tupac Shakur,"Ten pictures, from iconic shots to private sna...",https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, A Life in Ten Pictures, Seri...","Ten pictures, from iconic shots to private sna...",Tupac Shakur’s image is famous around the worl...,Tupac Shakur’s image is known around the world...,History,bbc_two,Contains some strong language.,10 Apr 2021,3531.0,"signed, history, documentaries",https://ichef.bbci.co.uk/images/ic/352x198/p09...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...
14750,A Life in Ten Pictures,Series 1: 1. Freddie Mercury,"Ten pictures, from iconic shots to private sna...",https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, A Life in Ten Pictures, Seri...","Ten pictures, from iconic shots to private sna...",Freddie Mercury’s image is famous around the w...,Freddie Mercury’s image is known around the wo...,History,bbc_two,Contains some strong language.,9pm 3 Apr 2021,3532.0,"signed, history, documentaries",https://ichef.bbci.co.uk/images/ic/352x198/p09...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...
17236,A Life in Ten Pictures,Series 1: 5. Muhammad Ali,"Ten pictures, from iconic shots to private sna...",https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, A Life in Ten Pictures, Seri...","Ten pictures, from iconic shots to private sna...",Muhammad Ali is one of the most photographed m...,Muhammad Ali is one of the most photographed m...,History,bbc_two,False,9pm 8 May 2021,3546.0,"signed, history, documentaries",https://ichef.bbci.co.uk/images/ic/352x198/p09...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...
17420,A Life in Ten Pictures,Series 1: 3. Elizabeth Taylor,"Ten pictures, from iconic shots to private sna...",https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, A Life in Ten Pictures, Seri...","Ten pictures, from iconic shots to private sna...",Elizabeth Taylor’s image is famous around the ...,Elizabeth Taylor’s image is known around the w...,History,bbc_two,Contains some strong language.,9:10pm 17 Apr 2021,3556.0,"signed, history, documentaries",https://ichef.bbci.co.uk/images/ic/352x198/p09...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...
18568,A Life in Ten Pictures,Series 1: 4. John Lennon,"Ten pictures, from iconic shots to private sna...",https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, A Life in Ten Pictures, Seri...","Ten pictures, from iconic shots to private sna...",John Lennon’s image is famous around the world...,John Lennon’s image is known around the world;...,History,bbc_two,False,9pm 15 May 2021,3539.0,"signed, history, documentaries",https://ichef.bbci.co.uk/images/ic/352x198/p09...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...


In [226]:
# df.to_csv('temp_bbc.csv', sep=';')

## Genre and Keyword extraction

In [231]:
categories = set(df['category'].tolist())
categories

{'Arts',
 'Beauty',
 'CBBC',
 'CBeebies',
 'Comedy',
 'Comedy Drama',
 'Crime Drama',
 'Current Affairs',
 'Dating',
 'Documentary',
 'Documentary Film',
 'Drama',
 'Entertainment',
 'Fantasy',
 'Film',
 'Film - Action',
 'Film - Adventure',
 'Film - Biopic',
 'Film - Comedy',
 'Film - Crime',
 'Film - Drama',
 'Film - Family',
 'Film - Fantasy',
 'Film - Horror',
 'Film - Musical',
 'Film - Period',
 'Film - Rom-Com',
 'Film - Romance',
 'Film - Sci-Fi',
 'Film - Thriller',
 'Film - War',
 'Film - Western',
 'Food',
 'From the Archives',
 'History',
 'Lifestyle',
 'Music',
 'Nature',
 'News',
 'Period Drama',
 'Reality',
 'Religion',
 'Rom-com',
 'Sci-Fi',
 'Science',
 'Science & Nature',
 'Signed',
 'Sport'}

In [230]:
df.loc[df['category'] == 'signed', 'category'] = 'Signed'
df.loc[df['category'] == 'science-and-nature', 'category'] = 'Science & Nature'
df.loc[df['category'] == 'news', 'category'] = 'News'
df.loc[df['category'] == 'lifestyle', 'category'] = 'Lifestyle'
df.loc[df['category'] == 'from-the-archives', 'category'] = 'From the Archives'
df.loc[df['category'] == 'food', 'category'] = 'Food'
df.loc[df['category'] == 'entertainment', 'category'] = 'Entertainment'
df.loc[df['category'] == 'documentaries', 'category'] = 'Documentary'
df.loc[df['category'] == 'cbbc', 'category'] = 'CBBC'
df.loc[df['category'] == 'arts', 'category'] = 'Arts'
df.loc[df['category'] == 'Sci-fi', 'category'] = 'Sci-Fi'

In [232]:
df[df['category'] == 'Film - Adventure']

,title,subtitle,description,image,keywords,syno_s,syno_m,syno_l,category,channel,language,release_date,duration_sec,topic,image_l,image_xl
1788,Ivanhoe,None,Ivanhoe fights for the honour of the kidnapped...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Ivanhoe",Ivanhoe fights for the honour of the kidnapped...,Swashbuckling spectacular based on the novel b...,Swashbuckling spectacular based on the novel b...,Film - Adventure,bbc_two,False,1952,6136.0,films,https://ichef.bbci.co.uk/images/ic/352x198/p0b...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...
1811,Lara Croft: Tomb Raider,None,The heroine races against time to find a relic...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Lara Croft: Tomb Raider",The heroine races against time to find a relic...,Action tale adapted from the popular video gam...,Action tale adapted from the popular video gam...,Film - Adventure,bbc_three,False,2001,5614.0,"films, drama-and-soaps",https://ichef.bbci.co.uk/images/ic/352x198/p07...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...
1819,El Cid,None,The epic story of the 11th-century Spanish her...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, El Cid",The epic story of the 11th-century Spanish her...,Epic story of the Spanish hero (Charlton Hesto...,The epic story of the 11th-century Spanish her...,Film - Adventure,bbc_two,False,1961,10297.0,"films, drama-and-soaps",https://ichef.bbci.co.uk/images/ic/352x198/p0b...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...
1841,The Spanish Main,None,Swashbuckling adventure in which a crew of ens...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, The Spanish Main",Swashbuckling adventure in which a crew of ens...,Swashbuckling adventure in which a Spanish gov...,Early RKO Technicolor swashbuckling adventure....,Film - Adventure,bbc_two,False,1945,5741.0,"films, drama-and-soaps",https://ichef.bbci.co.uk/images/ic/352x198/p08...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...
1849,Lara Croft Tomb Raider: The Cradle of Life,None,Archaeologist Lara Croft faces a race against ...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Lara Croft Tomb Raider: The ...",Archaeologist Lara Croft faces a race against ...,Globetrotting action sequel in which archaeolo...,Globetrotting action sequel in which intrepid ...,Film - Adventure,bbc_three,False,2003,6320.0,"films, drama-and-soaps",https://ichef.bbci.co.uk/images/ic/352x198/p08...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...
1863,Blackbeard the Pirate,None,Swashbuckling adventure set on the high seas i...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Blackbeard the Pirate",Swashbuckling adventure set on the high seas i...,Swashbuckler set on the high seas in the 17th ...,Swashbuckling adventure set on the high seas i...,Film - Adventure,bbc_two,False,1952,5913.0,"films, drama-and-soaps",https://ichef.bbci.co.uk/images/ic/352x198/p08...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...
9819,Around the World in 80 Days (Film),None,Phileas Fogg makes a wager that he can circumn...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Around the World in 80 Days ...",Phileas Fogg makes a wager that he can circumn...,Star-studded adventure yarn following the trav...,Reform Club member Phileas Fogg makes a wager ...,Film - Adventure,bbc_four,False,1956,9921.0,drama-and-soaps,https://ichef.bbci.co.uk/images/ic/352x198/p09...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...
10880,The Lost City of Z,None,"The story of Colonel Percy Fawcett, who led se...",https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, The Lost City of Z","The story of Colonel Percy Fawcett, who led se...","The true story of Colonel Percy Fawcett, who l...","The true story of Colonel Percy Fawcett, who l...",Film - Adventure,bbc_two,Contains some strong language and some violence.,2017,7680.0,drama-and-soaps,https://ichef.bbci.c

In [233]:
# df.to_csv('temp_bbc.csv', sep=';')

## Advanced Dataframe view

In [234]:
df

,title,subtitle,description,image,keywords,syno_s,syno_m,syno_l,category,channel,language,release_date,duration_sec,topic,image_l,image_xl
0,The 80s - Music’s Greatest Decade?,Series 1: 1. With Dylan Jones,Music author Dylan Jones argues that the 1980s...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, The 80s - Music’s Greatest D...",Music author Dylan Jones argues that the 1980s...,Music writer Dylan Jones puts the case that th...,Eighties music is often dismissed as a joke - ...,Music,bbc_two,False,8:55pm 23 Oct 2021,3534.0,"music, documentaries",https://ichef.bbci.co.uk/images/ic/352x198/p09...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...
1,Top of the Pops,02/07/1992,"With Arrested Development, Altern 8, Def Leppa...",https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Top of the Pops, 02/07/1992","With Arrested Development, Altern 8, Def Leppa...","Tony Dortie presents the pop chart programme, ...","Tony Dortie presents the pop chart programme, ...",Music,bbc_one,False,7pm 2 Jul 1992,1759.0,music,https://ichef.bbci.co.uk/images/ic/352x198/p0b...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...
2,BBC Proms,"2021: Last Night of the Proms, Part 2",Australian tenor Stuart Skelton is joined by L...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, BBC Proms, 2021: Last Night ...",Australian tenor Stuart Skelton is joined by L...,Australian tenor Stuart Skelton is joined by L...,Katie Derham hosts continued live coverage fro...,Music,bbc_radio_three,False,9pm 11 Sep 2021,5247.0,"music, from-the-archives",https://ichef.bbci.co.uk/images/ic/352x198/p09...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...
3,Port,Series 5: Episode 4,"Previously unseen music from the series Port, ...",https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Port, Series 5: Episode 4","Previously unseen music from the series Port, ...",Julie Fowlis a’ lìbhrigeadh ceòl bhon t-sreath...,Bidh Druthag Bheag Eile a’ tarraing ri chèile ...,Music,bbc_alba,False,9 Jul 2020,1037.0,music,https://ichef.bbci.co.uk/images/ic/352x198/p08...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...
4,"Sit Down, Stand Up with Greg James",None,Tom Holland and Greg James cold call celebrity...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Sit Down, Stand Up with Greg...",Tom Holland and Greg James cold call celebrity...,Tom Holland and Greg James cold call celebrity...,Tom Holland and Greg James cold call celebrity...,Entertainment,bbc_radio_one,False,10 Feb 2022,611.0,"music, entertainment",https://ichef.bbci.co.uk/images/ic/352x198/p0b...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36883,Bob the Builder,Series 8: 7. Ballroom Bob,Bob has been making secret early morning trips...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Bob the Builder, Series 8: 7...",Bob has been making secret early morning trips...,Bob has been making secret early morning trips...,Bob has been making secret early morning trips...,CBeebies,cbeebies,False,13 Oct 2003,600.0,cbeebees,https://ichef.bbci.co.uk/images/ic/352x198/p08...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...
36886,Go Buster,Stories: 5. Buster's Stories,Buster the bus has so many stories and adventu...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Go Buster, Stories: 5. Buste...",Buster the bus has so many stories and adventu...,Buster the bus has so many stories and adventu...,Buster the bus has so many stories and adventu...,CBeebies,cbeebies,False,18 Jun 2021,1580.0,cbeebees,https://ichef.bbci.co.uk/images/ic/352x198/p09...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...
36891,Chuggington,Badge Quest: 3. Rainbow Chuggers,The Chuggers need to solve a number of clues t...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Chuggington, Badge Quest: 3....",The Chuggers need to solve a number of clues

In [235]:
def reformat_date(val, date_month_year=True):
    val = str(val)
    if date_month_year:
        if len(val) == 10:
            date_str = val[:10]
        else:
            date_str = val[len(val)-11: len(val)]
    else:
        length = len(val)
        return val[length-4:length]
    date_time_obj = datetime.strptime(date_str, '%d %b %Y')
    return date_time_obj

In [237]:
df.loc[:, 'release_date'] = df['release_date'].apply(lambda x: reformat_date(x, False))

In [238]:
df

,title,subtitle,description,image,keywords,syno_s,syno_m,syno_l,category,channel,language,release_date,duration_sec,topic,image_l,image_xl
0,The 80s - Music’s Greatest Decade?,Series 1: 1. With Dylan Jones,Music author Dylan Jones argues that the 1980s...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, The 80s - Music’s Greatest D...",Music author Dylan Jones argues that the 1980s...,Music writer Dylan Jones puts the case that th...,Eighties music is often dismissed as a joke - ...,Music,bbc_two,False,2021,3534.0,"music, documentaries",https://ichef.bbci.co.uk/images/ic/352x198/p09...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...
1,Top of the Pops,02/07/1992,"With Arrested Development, Altern 8, Def Leppa...",https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Top of the Pops, 02/07/1992","With Arrested Development, Altern 8, Def Leppa...","Tony Dortie presents the pop chart programme, ...","Tony Dortie presents the pop chart programme, ...",Music,bbc_one,False,1992,1759.0,music,https://ichef.bbci.co.uk/images/ic/352x198/p0b...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...
2,BBC Proms,"2021: Last Night of the Proms, Part 2",Australian tenor Stuart Skelton is joined by L...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, BBC Proms, 2021: Last Night ...",Australian tenor Stuart Skelton is joined by L...,Australian tenor Stuart Skelton is joined by L...,Katie Derham hosts continued live coverage fro...,Music,bbc_radio_three,False,2021,5247.0,"music, from-the-archives",https://ichef.bbci.co.uk/images/ic/352x198/p09...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...
3,Port,Series 5: Episode 4,"Previously unseen music from the series Port, ...",https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Port, Series 5: Episode 4","Previously unseen music from the series Port, ...",Julie Fowlis a’ lìbhrigeadh ceòl bhon t-sreath...,Bidh Druthag Bheag Eile a’ tarraing ri chèile ...,Music,bbc_alba,False,2020,1037.0,music,https://ichef.bbci.co.uk/images/ic/352x198/p08...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...
4,"Sit Down, Stand Up with Greg James",None,Tom Holland and Greg James cold call celebrity...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Sit Down, Stand Up with Greg...",Tom Holland and Greg James cold call celebrity...,Tom Holland and Greg James cold call celebrity...,Tom Holland and Greg James cold call celebrity...,Entertainment,bbc_radio_one,False,2022,611.0,"music, entertainment",https://ichef.bbci.co.uk/images/ic/352x198/p0b...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36883,Bob the Builder,Series 8: 7. Ballroom Bob,Bob has been making secret early morning trips...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Bob the Builder, Series 8: 7...",Bob has been making secret early morning trips...,Bob has been making secret early morning trips...,Bob has been making secret early morning trips...,CBeebies,cbeebies,False,2003,600.0,cbeebees,https://ichef.bbci.co.uk/images/ic/352x198/p08...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...
36886,Go Buster,Stories: 5. Buster's Stories,Buster the bus has so many stories and adventu...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Go Buster, Stories: 5. Buste...",Buster the bus has so many stories and adventu...,Buster the bus has so many stories and adventu...,Buster the bus has so many stories and adventu...,CBeebies,cbeebies,False,2021,1580.0,cbeebees,https://ichef.bbci.co.uk/images/ic/352x198/p09...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...
36891,Chuggington,Badge Quest: 3. Rainbow Chuggers,The Chuggers need to solve a number of clues t...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Chuggington, Badge Quest: 3....",The Chuggers need to solve a number of clues t...,"Koko, Wilson and Brewster learn new skills and...","Kok

In [239]:
df.loc[:, 'duration_sec'] = df['duration_sec'].apply(lambda x: int(round(x/60, 0)))

In [240]:
df.loc[:, 'channel'] = df['channel'].apply(lambda x: x.upper())
df.loc[:, 'channel'] = df['channel'].apply(lambda x: x.replace('_', ' '))

In [ ]:
df.columns = ['title', 'subtitle', 'description', 'image', 'keywords', 'syno_s',
       'syno_m', 'syno_l', 'category', 'channel', 'language', 'release_date',
       'duration_min', 'topic', 'image_l', 'image_xl']

## Episode and Series

In [268]:
df = pd.read_csv('temp_bbc.csv', sep=';', index_col=0)
df

,title,subtitle,description,image,keywords,syno_s,syno_m,syno_l,category,channel,language,release_date,duration_min,topic,image_l,image_xl
0,The 80s - Music’s Greatest Decade?,Series 1: 1. With Dylan Jones,Music author Dylan Jones argues that the 1980s...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, The 80s - Music’s Greatest D...",Music author Dylan Jones argues that the 1980s...,Music writer Dylan Jones puts the case that th...,Eighties music is often dismissed as a joke - ...,Music,BBC TWO,False,2021,59,"music, documentaries",https://ichef.bbci.co.uk/images/ic/352x198/p09...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...
1,Top of the Pops,02/07/1992,"With Arrested Development, Altern 8, Def Leppa...",https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Top of the Pops, 02/07/1992","With Arrested Development, Altern 8, Def Leppa...","Tony Dortie presents the pop chart programme, ...","Tony Dortie presents the pop chart programme, ...",Music,BBC ONE,False,1992,29,music,https://ichef.bbci.co.uk/images/ic/352x198/p0b...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...
2,BBC Proms,"2021: Last Night of the Proms, Part 2",Australian tenor Stuart Skelton is joined by L...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, BBC Proms, 2021: Last Night ...",Australian tenor Stuart Skelton is joined by L...,Australian tenor Stuart Skelton is joined by L...,Katie Derham hosts continued live coverage fro...,Music,BBC RADIO THREE,False,2021,87,"music, from-the-archives",https://ichef.bbci.co.uk/images/ic/352x198/p09...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...
3,Port,Series 5: Episode 4,"Previously unseen music from the series Port, ...",https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Port, Series 5: Episode 4","Previously unseen music from the series Port, ...",Julie Fowlis a’ lìbhrigeadh ceòl bhon t-sreath...,Bidh Druthag Bheag Eile a’ tarraing ri chèile ...,Music,BBC ALBA,False,2020,17,music,https://ichef.bbci.co.uk/images/ic/352x198/p08...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...
4,"Sit Down, Stand Up with Greg James",None,Tom Holland and Greg James cold call celebrity...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Sit Down, Stand Up with Greg...",Tom Holland and Greg James cold call celebrity...,Tom Holland and Greg James cold call celebrity...,Tom Holland and Greg James cold call celebrity...,Entertainment,BBC RADIO ONE,False,2022,10,"music, entertainment",https://ichef.bbci.co.uk/images/ic/352x198/p0b...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36883,Bob the Builder,Series 8: 7. Ballroom Bob,Bob has been making secret early morning trips...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Bob the Builder, Series 8: 7...",Bob has been making secret early morning trips...,Bob has been making secret early morning trips...,Bob has been making secret early morning trips...,CBeebies,CBEEBIES,False,2003,10,cbeebees,https://ichef.bbci.co.uk/images/ic/352x198/p08...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...
36886,Go Buster,Stories: 5. Buster's Stories,Buster the bus has so many stories and adventu...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Go Buster, Stories: 5. Buste...",Buster the bus has so many stories and adventu...,Buster the bus has so many stories and adventu...,Buster the bus has so many stories and adventu...,CBeebies,CBEEBIES,False,2021,26,cbeebees,https://ichef.bbci.co.uk/images/ic/352x198/p09...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...
36891,Chuggington,Badge Quest: 3. Rainbow Chuggers,The Chuggers need to solve a number of clues t...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Chuggington, Badge Quest: 3....",The Chuggers need to solve a number of clues t...,"Koko, Wilson and Brewster learn new skills and...","Koko, Wilson and Brewster lea

In [253]:
filter_series = [row.split(':', 1) for row in df['subtitle'] if row.startswith('Series')]
filter_series

[['Series 1', ' 1. With Dylan Jones'],
 ['Series 5', ' Episode 4'],
 ['Series 5', ' 3. 1978'],
 ['Series 3', ' 1. With Taylor Swift'],
 ['Series 1', ' 1. Foo Fighters and More: Live Lounge Special'],
 ['Series 2', ' Episode 2'],
 ['Series 2', ' 2. George Ezra, Christine and the Queens and more'],
 ['Series 1', ' Episode 3'],
 ['Series 1', ' Episode 1'],
 ['Series 5', ' Episode 11'],
 ['Series 1', ' Episode 1'],
 ['Series 4', ' 2. 1973'],
 ['Series 1', ' Episode 1'],
 ['Series 2', ' Episode 6'],
 ['Series 4', ' 3. 1976'],
 ['Series 2', ' Episode 5'],
 ['Series 1', ' 4. 1985'],
 ['Series 1', ' 3. Sweet Love'],
 ['Series 1', ' Episode 6'],
 ['Series 1', ' 2. Say It Loud'],
 ['Series 5', ' Episode 10'],
 ['Series 3', ' 4. With Stormzy'],
 ['Series 3', ' Episode 6 - The Final'],
 ['Series 8', ' Episode 4'],
 ['Series 1', ' Episode 4'],
 ['Series 3', ' Episode 5'],
 ['Series 1', ' 1. Amazing Grace'],
 ['Series 1', ' 2. Alright, by Kendrick Lamar'],
 ['Series 4', ' 7. 1992'],
 ['Series 1', ' 

In [266]:
df

,title,subtitle,description,image,keywords,syno_s,syno_m,syno_l,category,channel,language,release_date,duration_min,topic,image_l,image_xl,first_part,second_part,series,episode
0,The 80s - Music’s Greatest Decade?,Series 1: 1. With Dylan Jones,Music author Dylan Jones argues that the 1980s...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, The 80s - Music’s Greatest D...",Music author Dylan Jones argues that the 1980s...,Music writer Dylan Jones puts the case that th...,Eighties music is often dismissed as a joke - ...,Music,BBC TWO,False,2021,59,"music, documentaries",https://ichef.bbci.co.uk/images/ic/352x198/p09...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,Series 1,1. With Dylan Jones,Series 1,1. With Dylan Jones
1,Top of the Pops,02/07/1992,"With Arrested Development, Altern 8, Def Leppa...",https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Top of the Pops, 02/07/1992","With Arrested Development, Altern 8, Def Leppa...","Tony Dortie presents the pop chart programme, ...","Tony Dortie presents the pop chart programme, ...",Music,BBC ONE,False,1992,29,music,https://ichef.bbci.co.uk/images/ic/352x198/p0b...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,None,None,None,None
2,BBC Proms,"2021: Last Night of the Proms, Part 2",Australian tenor Stuart Skelton is joined by L...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, BBC Proms, 2021: Last Night ...",Australian tenor Stuart Skelton is joined by L...,Australian tenor Stuart Skelton is joined by L...,Katie Derham hosts continued live coverage fro...,Music,BBC RADIO THREE,False,2021,87,"music, from-the-archives",https://ichef.bbci.co.uk/images/ic/352x198/p09...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,2021,"Last Night of the Proms, Part 2",None,None
3,Port,Series 5: Episode 4,"Previously unseen music from the series Port, ...",https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Port, Series 5: Episode 4","Previously unseen music from the series Port, ...",Julie Fowlis a’ lìbhrigeadh ceòl bhon t-sreath...,Bidh Druthag Bheag Eile a’ tarraing ri chèile ...,Music,BBC ALBA,False,2020,17,music,https://ichef.bbci.co.uk/images/ic/352x198/p08...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,Series 5,Episode 4,Series 5,Episode 4
4,"Sit Down, Stand Up with Greg James",None,Tom Holland and Greg James cold call celebrity...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Sit Down, Stand Up with Greg...",Tom Holland and Greg James cold call celebrity...,Tom Holland and Greg James cold call celebrity...,Tom Holland and Greg James cold call celebrity...,Entertainment,BBC RADIO ONE,False,2022,10,"music, entertainment",https://ichef.bbci.co.uk/images/ic/352x198/p0b...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36883,Bob the Builder,Series 8: 7. Ballroom Bob,Bob has been making secret early morning trips...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Bob the Builder, Series 8: 7...",Bob has been making secret early morning trips...,Bob has been making secret early morning trips...,Bob has been making secret early morning trips...,CBeebies,CBEEBIES,False,2003,10,cbeebees,https://ichef.bbci.co.uk/images/ic/352x198/p08...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,Series 8,7. Ballroom Bob,Series 8,7. Ballroom Bob
36886,Go Buster,Stories: 5. Buster's Stories,Buster the bus has so many stories and adventu...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Go Buster, Stories: 5. Buste...",Buster the bus has so many stories and adventu...,Buster the bus has so many stories and adventu...,Buster the bus has so many stories and adventu...,CBeebies,CBEEBIES,False,2021,26,cbeebees,https://ichef.bbci.co.uk/images/ic/352x198/p09...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,Stories,5. Buster's Stories,None,None
36891,Ch

In [269]:
import re
def extract_series_episode(df):
    df['series'] = 'None'
    df['episode'] = 'None'
    for i, entry in df.iterrows():
        if re.match('Series*:*', entry['subtitle']):
            split_array = entry['subtitle'].split(':', 1)
            df.loc[i, 'series'] = split_array[0]
            df.loc[i, 'episode'] = split_array[1]
extract_series_episode(df)

In [270]:
df

,title,subtitle,description,image,keywords,syno_s,syno_m,syno_l,category,channel,language,release_date,duration_min,topic,image_l,image_xl,series,episode
0,The 80s - Music’s Greatest Decade?,Series 1: 1. With Dylan Jones,Music author Dylan Jones argues that the 1980s...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, The 80s - Music’s Greatest D...",Music author Dylan Jones argues that the 1980s...,Music writer Dylan Jones puts the case that th...,Eighties music is often dismissed as a joke - ...,Music,BBC TWO,False,2021,59,"music, documentaries",https://ichef.bbci.co.uk/images/ic/352x198/p09...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,Series 1,1. With Dylan Jones
1,Top of the Pops,02/07/1992,"With Arrested Development, Altern 8, Def Leppa...",https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Top of the Pops, 02/07/1992","With Arrested Development, Altern 8, Def Leppa...","Tony Dortie presents the pop chart programme, ...","Tony Dortie presents the pop chart programme, ...",Music,BBC ONE,False,1992,29,music,https://ichef.bbci.co.uk/images/ic/352x198/p0b...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,None,None
2,BBC Proms,"2021: Last Night of the Proms, Part 2",Australian tenor Stuart Skelton is joined by L...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, BBC Proms, 2021: Last Night ...",Australian tenor Stuart Skelton is joined by L...,Australian tenor Stuart Skelton is joined by L...,Katie Derham hosts continued live coverage fro...,Music,BBC RADIO THREE,False,2021,87,"music, from-the-archives",https://ichef.bbci.co.uk/images/ic/352x198/p09...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,None,None
3,Port,Series 5: Episode 4,"Previously unseen music from the series Port, ...",https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Port, Series 5: Episode 4","Previously unseen music from the series Port, ...",Julie Fowlis a’ lìbhrigeadh ceòl bhon t-sreath...,Bidh Druthag Bheag Eile a’ tarraing ri chèile ...,Music,BBC ALBA,False,2020,17,music,https://ichef.bbci.co.uk/images/ic/352x198/p08...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,Series 5,Episode 4
4,"Sit Down, Stand Up with Greg James",None,Tom Holland and Greg James cold call celebrity...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Sit Down, Stand Up with Greg...",Tom Holland and Greg James cold call celebrity...,Tom Holland and Greg James cold call celebrity...,Tom Holland and Greg James cold call celebrity...,Entertainment,BBC RADIO ONE,False,2022,10,"music, entertainment",https://ichef.bbci.co.uk/images/ic/352x198/p0b...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36883,Bob the Builder,Series 8: 7. Ballroom Bob,Bob has been making secret early morning trips...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Bob the Builder, Series 8: 7...",Bob has been making secret early morning trips...,Bob has been making secret early morning trips...,Bob has been making secret early morning trips...,CBeebies,CBEEBIES,False,2003,10,cbeebees,https://ichef.bbci.co.uk/images/ic/352x198/p08...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,Series 8,7. Ballroom Bob
36886,Go Buster,Stories: 5. Buster's Stories,Buster the bus has so many stories and adventu...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Go Buster, Stories: 5. Buste...",Buster the bus has so many stories and adventu...,Buster the bus has so many stories and adventu...,Buster the bus has so many stories and adventu...,CBeebies,CBEEBIES,False,2021,26,cbeebees,https://ichef.bbci.co.uk/images/ic/352x198/p09...,https://ichef.bbci.co.uk/images/ic/1200x675/p0...,None,None
36891,Chuggington,Badge Quest: 3. Rainbow Chuggers,The Chuggers need to solve a number of clues t...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Chuggington, Badge Quest: 3..

In [272]:
df.to_csv('bbc.csv', sep=';')

## TF-IDF

In [4]:
df['mixed_text'] = df['title'] + ' ' + df['description'] + ' ' + df['syno_l'] + ' ' + df['category']

In [5]:
import nltk
from nltk.corpus import stopwords
lemmatizer = nltk.stem.WordNetLemmatizer()

def clean_text(x, stop_words):
  if pd.isna(x) == False:
    x = x.lower()
    x = x.translate(str.maketrans(' ', ' ', string.punctuation))
    x = x.strip()

    x = nltk.word_tokenize(x)
    x = [token for token in x if not token in stop_words]
    x = ' '.join([lemmatizer.lemmatize(w) for w in x])
  else:
    x = ''

  return x

In [6]:
stop_words = stopwords.words('english')
df['cleaned_mixed_text'] = df['mixed_text'].apply(lambda x: clean_text(x, stop_words))

In [7]:
df['cleaned_mixed_text']

0       bbc prom australian tenor stuart skelton joine...
1       port previously unseen music series port julie...
2       sit stand greg james tom holland greg james co...
3       ceiliúradh na féile pádraig john toal pauline ...
4       song praise claire mccollum explores edinburgh...
                              ...                        
3777    model street catwalk kyra ’ late arrival put m...
3789    death i95 returning u find happened people met...
3793    black life still matter grime artist saskilla ...
3797    ar sliabh paul pack return native armagh siún ...
3800    avicii true story avicii true story tim bergli...
Name: cleaned_mixed_text, Length: 2326, dtype: object

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn import cluster

vectorizer = TfidfVectorizer(min_df=3, max_df=0.80, norm='l2')
X = vectorizer.fit_transform(df['cleaned_mixed_text'])
tf_idf = pd.DataFrame(data = X.toarray(), columns=vectorizer.get_feature_names())
tf_idf

/Users/hagenaadfock/miniforge3/envs/default/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,10,100,1000,10000,100000,100th,107,10th,11,12,...,às,áitiúil,òg,òga,òrain,òran,óg,ùr,ùra,údar
0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
1,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.051456,0.000000,0.0,0.0,0.0,0.064423,0.124002,0.0,0.0,0.0
2,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
3,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
4,0.0,0.07281,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2321,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
2322,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
2323,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
2324,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.097653,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0


In [ ]:
clusters = 3
kmeanModel = KMeans(n_clusters=clusters, init='k-means++', max_iter=3000, random_state=0)
mod = kmeanModel.fit_transform(tf_idf)
df['k_means'] = kmeanModel.predict(tf_idf)

In [ ]:
order_centroids = kmeanModel.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
dict = []
for i in range(clusters):
  print('%d' % i, sep='', end=', '),
  for ind in order_centroids[i, :20]:
    print(terms[ind], sep='', end=', ')
  print('')

In [ ]:
df[df['k_means'] == 2]['Title'].head(20)

## Cosin Similarity

In [6]:
df_cos = pd.read_csv('./../data/BBC/cosine_similarity.csv', sep=';', index_col=0)

In [7]:
df_cos

,0,1,2,3,4,5,6,7,8,9,...,2316,2317,2318,2319,2320,2321,2322,2323,2324,2325
0,1.000000,0.464569,0.456215,0.759457,0.752888,0.817963,0.844283,0.833293,0.624325,0.714856,...,0.494571,0.404984,0.297471,0.484163,0.610856,0.386949,0.408679,0.504979,0.394167,0.591685
1,0.464569,1.000000,0.276400,0.500826,0.409278,0.324603,0.260997,0.318547,0.332594,0.532213,...,0.237382,0.313001,0.229477,0.322347,0.442119,0.175973,0.237692,0.267875,0.877725,0.278713
2,0.456215,0.276400,1.000000,0.515068,0.463887,0.565514,0.456465,0.480797,0.404655,0.434049,...,0.509228,0.466864,0.372347,0.358217,0.302932,0.401385,0.337785,0.541091,0.310047,0.442695
3,0.759457,0.500826,0.515068,1.000000,0.622449,0.738313,0.668563,0.714047,0.395446,0.621673,...,0.470608,0.398964,0.345343,0.394587,0.494136,0.348407,0.325590,0.466440,0.511251,0.503900
4,0.752888,0.409278,0.463887,0.622449,1.000000,0.705579,0.671373,0.769527,0.730229,0.689653,...,0.576755,0.506818,0.439136,0.587217,0.748822,0.580583,0.526698,0.634509,0.369326,0.638660
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2321,0.386949,0.175973,0.401385,0.348407,0.580583,0.467882,0.473950,0.493751,0.578729,0.474601,...,0.512476,0.594405,0.491411,0.708732,0.403681,1.000000,0.689001,0.661167,0.149656,0.557155
2322,0.408679,0.237692,0.337785,0.325590,0.526698,0.464923,0.516740,0.520363,0.595682,0.404929,...,0.571575,0.624707,0.529199,0.794125,0.304786,0.689001,1.000000,0.727510,0.162182,0.680074
2323,0.504979,0.267875,0.541091,0.466440,0.634509,0.596023,0.577080,0.571917,0.621068,0.463083,...,0.581656,0.532636,0.577583,0.712539,0.392916,0.661167,0.727510,1.000000,0.212749,0.643721
2324,0.394167,0.877725,0.310047,0.511251,0.369326,0.296917,0.209242,0.249882,0.257936,0.496477,...,0.179157,0.238167,0.201336,0.264322,0.384121,0.149656,0.162182,0.212749,1.000000,0.203437


## Slimmed bbc_data_view

In [20]:
df_bbc = pd.read_csv('./../data/BBC/bbc_data.csv', sep=';', index_col=0)
df_img = pd.read_csv('./../data/BBC/bbc_images.csv', sep=';', index_col=0)

In [21]:
df_bbc.head()

,index,title,description,image,keywords,syno_s,syno_m,syno_l,category,channel,language,release_date,duration_sec,topic,spacied_syno_l,processed_syno_l,processed_syno_l_one_string,k_means
0,0,BBC Proms,Australian tenor Stuart Skelton is joined by L...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, BBC Proms, 2021: Last Night ...",Australian tenor Stuart Skelton is joined by L...,Australian tenor Stuart Skelton is joined by L...,Katie Derham hosts continued live coverage fro...,Music,bbc_radio_three,False,9pm 11 Sep 2021,5247,"['music', 'from-the-archives']",Katie Derham hosts continued live coverage fro...,"['Katie', 'Derham', 'host', 'continue', 'live'...",Katie Derham host continue live coverage Royal...,11
1,1,Port,"Previously unseen music from the series Port, ...",https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Port, Series 5: Episode 4","Previously unseen music from the series Port, ...",Julie Fowlis a’ lìbhrigeadh ceòl bhon t-sreath...,Bidh Druthag Bheag Eile a’ tarraing ri chèile ...,Music,bbc_alba,False,9 Jul 2020,1037,['music'],Bidh Druthag Bheag Eile a’ tarraing ri chèile ...,"['Bidh', 'Druthag', 'Bheag', 'Eile', 'tarraing...",Bidh Druthag Bheag Eile tarraing ri chèile pìo...,7
2,2,"Sit Down, Stand Up with Greg James",Tom Holland and Greg James cold call celebrity...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Sit Down, Stand Up with Greg...",Tom Holland and Greg James cold call celebrity...,Tom Holland and Greg James cold call celebrity...,Tom Holland and Greg James cold call celebrity...,Entertainment,bbc_radio_one,False,10 Feb 2022,611,"['music', 'entertainment']",Tom Holland and Greg James cold call celebrity...,"['Tom', 'Holland', 'Greg', 'James', 'cold', 'c...",Tom Holland Greg James cold celebrity friend f...,10
3,3,Ceiliúradh na Féile Pádraig,John Toal and Pauline Scanlon introduce an eve...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Ceiliúradh na Féile Pádraig",John Toal and Pauline Scanlon introduce an eve...,John Toal and Pauline Scanlon introduce an eve...,John Toal and Pauline Scanlon introduce an eve...,Music,bbc_two_northern_ireland_digital,False,17 Mar 2017,3538,['music'],John Toal and Pauline Scanlon introduce an eve...,"['John', 'Toal', 'Pauline', 'Scanlon', 'introd...",John Toal Pauline Scanlon introduce evening mu...,7
4,4,Songs of Praise,Claire McCollum explores Edinburgh and shares ...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Songs of Praise, Edinburgh",Claire McCollum explores Edinburgh and shares ...,Claire McCollum goes on a Christian heritage t...,Claire McCollum explores Edinburgh and discove...,Music,bbc_one,False,1:15pm 20 Feb 2022,2044,"['music', 'lifestyle']",Claire McCollum explores Edinburgh and discove...,"['Claire', 'McCollum', 'explore', 'Edinburgh',...",Claire McCollum explore Edinburgh discover ric...,9


In [22]:
df_img.head()

,index,image,image_s,image_m,image_l
0,0,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,https://ichef.bbci.co.uk/images/ic/192x108/p09...,https://ichef.bbci.co.uk/images/ic/288x162/p09...,https://ichef.bbci.co.uk/images/ic/352x198/p09...
1,1,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,https://ichef.bbci.co.uk/images/ic/192x108/p08...,https://ichef.bbci.co.uk/images/ic/288x162/p08...,https://ichef.bbci.co.uk/images/ic/352x198/p08...
2,2,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,https://ichef.bbci.co.uk/images/ic/192x108/p0b...,https://ichef.bbci.co.uk/images/ic/288x162/p0b...,https://ichef.bbci.co.uk/images/ic/352x198/p0b...
3,3,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,https://ichef.bbci.co.uk/images/ic/192x108/p04...,https://ichef.bbci.co.uk/images/ic/288x162/p04...,https://ichef.bbci.co.uk/images/ic/352x198/p04...
4,4,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,https://ichef.bbci.co.uk/images/ic/192x108/p0b...,https://ichef.bbci.co.uk/images/ic/288x162/p0b...,https://ichef.bbci.co.uk/images/ic/352x198/p0b...


In [23]:
df_bbc_merg = pd.merge(df_bbc, df_img, on='index')

In [24]:
df_bbc_slim = df_bbc_merg.iloc[:, [0, 1, 2, 8, 9, 17, 21]]
df_bbc_slim

,index,title,description,category,channel,k_means,image_l
0,0,BBC Proms,Australian tenor Stuart Skelton is joined by L...,Music,bbc_radio_three,11,https://ichef.bbci.co.uk/images/ic/352x198/p09...
1,1,Port,"Previously unseen music from the series Port, ...",Music,bbc_alba,7,https://ichef.bbci.co.uk/images/ic/352x198/p08...
2,2,"Sit Down, Stand Up with Greg James",Tom Holland and Greg James cold call celebrity...,Entertainment,bbc_radio_one,10,https://ichef.bbci.co.uk/images/ic/352x198/p0b...
3,3,Ceiliúradh na Féile Pádraig,John Toal and Pauline Scanlon introduce an eve...,Music,bbc_two_northern_ireland_digital,7,https://ichef.bbci.co.uk/images/ic/352x198/p04...
4,4,Songs of Praise,Claire McCollum explores Edinburgh and shares ...,Music,bbc_one,9,https://ichef.bbci.co.uk/images/ic/352x198/p0b...
...,...,...,...,...,...,...,...
2321,3777,Models: Street to Catwalk,Kyra’s late arrival puts her modelling career ...,Documentary,bbc_three,9,https://ichef.bbci.co.uk/images/ic/352x198/p09...
2322,3789,Death On The I-95,Returning to the US to find out what happened ...,Documentary,bbc_three,1,https://ichef.bbci.co.uk/images/ic/352x198/p07...
2323,3793,Do Black Lives Still Matter?,"Grime artist Saskilla asks, can the music indu...",Documentary,bbc_three,11,https://ichef.bbci.co.uk/images/ic/352x198/p09...
2324,3797,Ar An Sliabh,Paul packs up to return to his native Armagh a...,Documentary,bbc_webonly,7,https://ichef.bbci.co.uk/images/ic/352x198/p0b...


In [25]:
df_bbc_slim.columns = ['index', 'title', 'description', 'category', 'channel', 'adv_category', 'img']

In [41]:
df_bbc_slim = df_bbc_slim.reset_index().iloc[:, [0, 2,3,4,5,6, 7]]

In [42]:
df_bbc_slim.columns = ['index', 'title', 'description', 'category', 'channel', 'adv_category', 'img']

In [43]:
# df_bbc_slim.to_csv('./../data/BBC/bbc_data_slim.csv')

In [77]:
combined = df_cos.iloc[:,[1,2,3]]
df_cos['combined'] = combined.mean(axis=1)
df_cos['combined'].sort_values(ascending=False)

3       0.671965
172     0.639252
183     0.628866
749     0.625085
192     0.621512
          ...   
1639    0.229174
2080    0.211944
935     0.205230
2055    0.195554
1385    0.175631
Name: combined, Length: 2326, dtype: float64

In [86]:
df.iloc[df_img.iloc[1:100].index]

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,2316,2317,2318,2319,2320,2321,2322,2323,2324,2325
1,1,0.464569,1.000000,0.276400,0.500826,0.409278,0.324603,0.260997,0.318547,0.332594,...,0.237382,0.313001,0.229477,0.322347,0.442119,0.175973,0.237692,0.267875,0.877725,0.278713
2,2,0.456215,0.276400,1.000000,0.515068,0.463887,0.565514,0.456465,0.480797,0.404655,...,0.509228,0.466864,0.372347,0.358217,0.302932,0.401385,0.337785,0.541091,0.310047,0.442695
3,3,0.759457,0.500826,0.515068,1.000000,0.622449,0.738313,0.668563,0.714047,0.395446,...,0.470608,0.398964,0.345343,0.394587,0.494136,0.348407,0.325590,0.466440,0.511251,0.503900
4,4,0.752888,0.409278,0.463887,0.622449,1.000000,0.705579,0.671373,0.769527,0.730229,...,0.576755,0.506818,0.439136,0.587217,0.748822,0.580583,0.526698,0.634509,0.369326,0.638660
5,5,0.817963,0.324603,0.565514,0.738313,0.705579,1.000000,0.853889,0.792876,0.535997,...,0.557160,0.461945,0.336006,0.531765,0.459272,0.467882,0.464923,0.596023,0.296917,0.703094
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,0.650923,0.324623,0.458271,0.536036,0.728032,0.712740,0.723246,0.789703,0.699495,...,0.573552,0.506305,0.389653,0.661572,0.553825,0.572377,0.643866,0.615163,0.261735,0.823727
96,96,0.609318,0.285616,0.438013,0.532466,0.698091,0.641838,0.629867,0.699402,0.644056,...,0.488548,0.369026,0.362290,0.524622,0.545288,0.472625,0.571902,0.614720,0.247530,0.672521
97,97,0.737199,0.291181,0.498639,0.608746,0.720109,0.864829,0.825857,0.818352,0.657735,...,0.679491,0.570903,0.383951,0.659221,0.476024,0.589684,0.632455,0.652837,0.223825,0.830106
98,98,0.750932,0.635492,0.430954,0.715362,0.635712,0.664503,0.649895,0.662768,0.471371,...,0.372842,0.410879,0.306685,0.407787,0.514606,0.395784,0.371811,0.401731,0.639460,0.483331


In [89]:
df

,index,title,description,image,keywords,syno_s,syno_m,syno_l,category,channel,language,release_date,duration_sec,topic,spacied_syno_l,processed_syno_l,processed_syno_l_one_string,k_means
0,0,BBC Proms,Australian tenor Stuart Skelton is joined by L...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, BBC Proms, 2021: Last Night ...",Australian tenor Stuart Skelton is joined by L...,Australian tenor Stuart Skelton is joined by L...,Katie Derham hosts continued live coverage fro...,Music,bbc_radio_three,False,9pm 11 Sep 2021,5247,"['music', 'from-the-archives']",Katie Derham hosts continued live coverage fro...,"['Katie', 'Derham', 'host', 'continue', 'live'...",Katie Derham host continue live coverage Royal...,11
1,1,Port,"Previously unseen music from the series Port, ...",https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Port, Series 5: Episode 4","Previously unseen music from the series Port, ...",Julie Fowlis a’ lìbhrigeadh ceòl bhon t-sreath...,Bidh Druthag Bheag Eile a’ tarraing ri chèile ...,Music,bbc_alba,False,9 Jul 2020,1037,['music'],Bidh Druthag Bheag Eile a’ tarraing ri chèile ...,"['Bidh', 'Druthag', 'Bheag', 'Eile', 'tarraing...",Bidh Druthag Bheag Eile tarraing ri chèile pìo...,7
2,2,"Sit Down, Stand Up with Greg James",Tom Holland and Greg James cold call celebrity...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Sit Down, Stand Up with Greg...",Tom Holland and Greg James cold call celebrity...,Tom Holland and Greg James cold call celebrity...,Tom Holland and Greg James cold call celebrity...,Entertainment,bbc_radio_one,False,10 Feb 2022,611,"['music', 'entertainment']",Tom Holland and Greg James cold call celebrity...,"['Tom', 'Holland', 'Greg', 'James', 'cold', 'c...",Tom Holland Greg James cold celebrity friend f...,10
3,3,Ceiliúradh na Féile Pádraig,John Toal and Pauline Scanlon introduce an eve...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Ceiliúradh na Féile Pádraig",John Toal and Pauline Scanlon introduce an eve...,John Toal and Pauline Scanlon introduce an eve...,John Toal and Pauline Scanlon introduce an eve...,Music,bbc_two_northern_ireland_digital,False,17 Mar 2017,3538,['music'],John Toal and Pauline Scanlon introduce an eve...,"['John', 'Toal', 'Pauline', 'Scanlon', 'introd...",John Toal Pauline Scanlon introduce evening mu...,7
4,4,Songs of Praise,Claire McCollum explores Edinburgh and shares ...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Songs of Praise, Edinburgh",Claire McCollum explores Edinburgh and shares ...,Claire McCollum goes on a Christian heritage t...,Claire McCollum explores Edinburgh and discove...,Music,bbc_one,False,1:15pm 20 Feb 2022,2044,"['music', 'lifestyle']",Claire McCollum explores Edinburgh and discove...,"['Claire', 'McCollum', 'explore', 'Edinburgh',...",Claire McCollum explore Edinburgh discover ric...,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3777,3777,Models: Street to Catwalk,Kyra’s late arrival puts her modelling career ...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Models: Street to Catwalk, S...",Kyra’s late arrival puts her modelling career ...,Kyra jeopardises her modelling career by turni...,How will Kyra cope after a train malfunction m...,Documentary,bbc_three,Contains some strong language and some upsetti...,8 Aug 2021,1843,['documentaries'],How will Kyra cope after a train malfunction m...,"['Kyra', 'cope', 'train', 'malfunction', 'make...",Kyra cope train malfunction make hour crucial ...,9
3789,3789,Death On The I-95,Returning to the US to find out what happened ...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Death On The I-95",Returning to the US to find out what happened ...,Following a report on the growing problem of o...,Drug overdose is now the main cause of death f...,Documentary,bbc_three,Contains some strong language and upsetting sc...,24 Oct 2018,342

## Messing around with similarity measures

In [2]:
df_bbc = pd.read_csv('./../data/BBC/bbc_data_slim.csv', sep=',', index_col=0)
cos = pd.read_csv('./../data/BBC/cosine_similarity.csv', sep=';', index_col=0)

In [5]:
df_bbc

,index,title,description,category,channel,adv_category,img
0,0,BBC Proms,Australian tenor Stuart Skelton is joined by L...,Music,bbc_radio_three,11,https://ichef.bbci.co.uk/images/ic/352x198/p09...
1,1,Port,"Previously unseen music from the series Port, ...",Music,bbc_alba,7,https://ichef.bbci.co.uk/images/ic/352x198/p08...
2,2,"Sit Down, Stand Up with Greg James",Tom Holland and Greg James cold call celebrity...,Entertainment,bbc_radio_one,10,https://ichef.bbci.co.uk/images/ic/352x198/p0b...
3,3,Ceiliúradh na Féile Pádraig,John Toal and Pauline Scanlon introduce an eve...,Music,bbc_two_northern_ireland_digital,7,https://ichef.bbci.co.uk/images/ic/352x198/p04...
4,4,Songs of Praise,Claire McCollum explores Edinburgh and shares ...,Music,bbc_one,9,https://ichef.bbci.co.uk/images/ic/352x198/p0b...
...,...,...,...,...,...,...,...
2321,2321,Models: Street to Catwalk,Kyra’s late arrival puts her modelling career ...,Documentary,bbc_three,9,https://ichef.bbci.co.uk/images/ic/352x198/p09...
2322,2322,Death On The I-95,Returning to the US to find out what happened ...,Documentary,bbc_three,1,https://ichef.bbci.co.uk/images/ic/352x198/p07...
2323,2323,Do Black Lives Still Matter?,"Grime artist Saskilla asks, can the music indu...",Documentary,bbc_three,11,https://ichef.bbci.co.uk/images/ic/352x198/p09...
2324,2324,Ar An Sliabh,Paul packs up to return to his native Armagh a...,Documentary,bbc_webonly,7,https://ichef.bbci.co.uk/images/ic/352x198/p0b...


In [6]:
cos

,0,1,2,3,4,5,6,7,8,9,...,2316,2317,2318,2319,2320,2321,2322,2323,2324,2325
0,1.000000,0.464569,0.456215,0.759457,0.752888,0.817963,0.844283,0.833293,0.624325,0.714856,...,0.494571,0.404984,0.297471,0.484163,0.610856,0.386949,0.408679,0.504979,0.394167,0.591685
1,0.464569,1.000000,0.276400,0.500826,0.409278,0.324603,0.260997,0.318547,0.332594,0.532213,...,0.237382,0.313001,0.229477,0.322347,0.442119,0.175973,0.237692,0.267875,0.877725,0.278713
2,0.456215,0.276400,1.000000,0.515068,0.463887,0.565514,0.456465,0.480797,0.404655,0.434049,...,0.509228,0.466864,0.372347,0.358217,0.302932,0.401385,0.337785,0.541091,0.310047,0.442695
3,0.759457,0.500826,0.515068,1.000000,0.622449,0.738313,0.668563,0.714047,0.395446,0.621673,...,0.470608,0.398964,0.345343,0.394587,0.494136,0.348407,0.325590,0.466440,0.511251,0.503900
4,0.752888,0.409278,0.463887,0.622449,1.000000,0.705579,0.671373,0.769527,0.730229,0.689653,...,0.576755,0.506818,0.439136,0.587217,0.748822,0.580583,0.526698,0.634509,0.369326,0.638660
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2321,0.386949,0.175973,0.401385,0.348407,0.580583,0.467882,0.473950,0.493751,0.578729,0.474601,...,0.512476,0.594405,0.491411,0.708732,0.403681,1.000000,0.689001,0.661167,0.149656,0.557155
2322,0.408679,0.237692,0.337785,0.325590,0.526698,0.464923,0.516740,0.520363,0.595682,0.404929,...,0.571575,0.624707,0.529199,0.794125,0.304786,0.689001,1.000000,0.727510,0.162182,0.680074
2323,0.504979,0.267875,0.541091,0.466440,0.634509,0.596023,0.577080,0.571917,0.621068,0.463083,...,0.581656,0.532636,0.577583,0.712539,0.392916,0.661167,0.727510,1.000000,0.212749,0.643721
2324,0.394167,0.877725,0.310047,0.511251,0.369326,0.296917,0.209242,0.249882,0.257936,0.496477,...,0.179157,0.238167,0.201336,0.264322,0.384121,0.149656,0.162182,0.212749,1.000000,0.203437


In [5]:
def sim_title_by_genre(df, df_cosin_similarity, index, number_of_recom, genre):
    fitting_genre = df[df['category'] == genre]
    return fitting_genre
    # print(len(fitting_genre))
    # df_cosin_genre_exc = df_cosin_similarity.iloc[[fitting_genre.index, index]]
    # recoms = df_cosin_genre_exc.loc[index].sort_values(ascending=False).index.tolist()[1:number_of_recom]
    # return list(map(int, recoms))

In [ ]:
sim_title_by_genre(df_bbc, cos, 3, 8, genre='Entertainment')

127


/Users/hagenaadfock/miniforge3/envs/default/lib/python3.8/site-packages/pandas/core/indexing.py:1503: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  key = np.asarray(key)


In [44]:
df_bbc

#cos

,index,title,description,category,channel,adv_category,img
0,0,BBC Proms,Australian tenor Stuart Skelton is joined by L...,Music,bbc_radio_three,11,https://ichef.bbci.co.uk/images/ic/352x198/p09...
1,1,Port,"Previously unseen music from the series Port, ...",Music,bbc_alba,7,https://ichef.bbci.co.uk/images/ic/352x198/p08...
2,2,"Sit Down, Stand Up with Greg James",Tom Holland and Greg James cold call celebrity...,Entertainment,bbc_radio_one,10,https://ichef.bbci.co.uk/images/ic/352x198/p0b...
3,3,Ceiliúradh na Féile Pádraig,John Toal and Pauline Scanlon introduce an eve...,Music,bbc_two_northern_ireland_digital,7,https://ichef.bbci.co.uk/images/ic/352x198/p04...
4,4,Songs of Praise,Claire McCollum explores Edinburgh and shares ...,Music,bbc_one,9,https://ichef.bbci.co.uk/images/ic/352x198/p0b...
...,...,...,...,...,...,...,...
2321,2321,Models: Street to Catwalk,Kyra’s late arrival puts her modelling career ...,Documentary,bbc_three,9,https://ichef.bbci.co.uk/images/ic/352x198/p09...
2322,2322,Death On The I-95,Returning to the US to find out what happened ...,Documentary,bbc_three,1,https://ichef.bbci.co.uk/images/ic/352x198/p07...
2323,2323,Do Black Lives Still Matter?,"Grime artist Saskilla asks, can the music indu...",Documentary,bbc_three,11,https://ichef.bbci.co.uk/images/ic/352x198/p09...
2324,2324,Ar An Sliabh,Paul packs up to return to his native Armagh a...,Documentary,bbc_webonly,7,https://ichef.bbci.co.uk/images/ic/352x198/p0b...


In [46]:
fitting_genre = df_bbc[df_bbc['category'] == 'Music']
cosin_df = cos.loc[0].iloc[fitting_genre.index].sort_values(ascending=False)[1:6]
cosin_df
# cosin_df = cos.loc[1].sort_values(ascending=False)
# cosin_df = cosin_df.iloc[fitting_genre.index]
# cosin_df.index
# recoms = cosin_df[1:6].index.tolist()
# recoms

6      0.844283
48     0.840854
134    0.840419
179    0.840170
85     0.839027
Name: 0, dtype: float64

In [23]:
df_bbc.iloc[cosin_df.index]

,index,title,description,category,channel,adv_category,img
1770,1770,Politics Live,Jo Coburn and guests discuss the military assa...,News,bbc_two,10,https://ichef.bbci.co.uk/images/ic/352x198/p09...
1851,1851,The Data Guardian: Elizabeth Denham,BBC Media Editor Amol Rajan talks to UK Inform...,News,bbc_news24,13,https://ichef.bbci.co.uk/images/ic/352x198/p0b...
854,854,Tropic of Cancer with Simon Reeve,Simon travels from the jungles of Southeast As...,Documentary,bbc_two,6,https://ichef.bbci.co.uk/images/ic/352x198/p01...
1845,1845,Northern Ireland Assembly,"Tributes to the DUP MLA Christopher Stalford, ...",News,bbc_parliament,2,https://ichef.bbci.co.uk/images/ic/352x198/p0b...
295,295,Life Cinematic,Amma Asante selects clips from the films that ...,Arts,bbc_four,9,https://ichef.bbci.co.uk/images/ic/352x198/p09...
...,...,...,...,...,...,...,...
1332,1332,Rugby League: Challenge Cup,Third Round of the Challenge Cup live from the...,Sport,bbc_sport,2,https://ichef.bbci.co.uk/images/ic/352x198/p0b...
33,33,Canadian Hits at the BBC,A selection of performances by Canadian artist...,Music,bbc_four,11,https://ichef.bbci.co.uk/images/ic/352x198/p09...
1980,1980,Back in Time for the Corner Shop,The Arderns’ shopkeeping adventure continues i...,Food,bbc_two,4,https://ichef.bbci.co.uk/images/ic/352x198/p08...
1642,1642,The Grand Party Hotel,MD Kris is asked to fight in a charity boxing ...,Documentary,bbc_one,4,https://ichef.bbci.co.uk/images/ic/352x198/p08...


In [47]:
df_bbc

,index,title,description,category,channel,adv_category,img
0,0,BBC Proms,Australian tenor Stuart Skelton is joined by L...,Music,bbc_radio_three,11,https://ichef.bbci.co.uk/images/ic/352x198/p09...
1,1,Port,"Previously unseen music from the series Port, ...",Music,bbc_alba,7,https://ichef.bbci.co.uk/images/ic/352x198/p08...
2,2,"Sit Down, Stand Up with Greg James",Tom Holland and Greg James cold call celebrity...,Entertainment,bbc_radio_one,10,https://ichef.bbci.co.uk/images/ic/352x198/p0b...
3,3,Ceiliúradh na Féile Pádraig,John Toal and Pauline Scanlon introduce an eve...,Music,bbc_two_northern_ireland_digital,7,https://ichef.bbci.co.uk/images/ic/352x198/p04...
4,4,Songs of Praise,Claire McCollum explores Edinburgh and shares ...,Music,bbc_one,9,https://ichef.bbci.co.uk/images/ic/352x198/p0b...
...,...,...,...,...,...,...,...
2321,2321,Models: Street to Catwalk,Kyra’s late arrival puts her modelling career ...,Documentary,bbc_three,9,https://ichef.bbci.co.uk/images/ic/352x198/p09...
2322,2322,Death On The I-95,Returning to the US to find out what happened ...,Documentary,bbc_three,1,https://ichef.bbci.co.uk/images/ic/352x198/p07...
2323,2323,Do Black Lives Still Matter?,"Grime artist Saskilla asks, can the music indu...",Documentary,bbc_three,11,https://ichef.bbci.co.uk/images/ic/352x198/p09...
2324,2324,Ar An Sliabh,Paul packs up to return to his native Armagh a...,Documentary,bbc_webonly,7,https://ichef.bbci.co.uk/images/ic/352x198/p0b...


In [70]:
df_bbc[df_bbc['index'] == 5]['adv_category'].iloc[0]

11

In [64]:
df_bbc[df_bbc['adv_category'] == df_bbc[df_bbc['index'] == 1].iloc[0, 0]].index.tolist()

[247,
 324,
 434,
 473,
 509,
 533,
 537,
 541,
 593,
 597,
 599,
 613,
 619,
 621,
 625,
 691,
 706,
 708,
 717,
 742,
 748,
 778,
 791,
 792,
 808,
 809,
 811,
 814,
 825,
 826,
 827,
 851,
 858,
 863,
 868,
 871,
 873,
 874,
 880,
 897,
 904,
 907,
 918,
 921,
 923,
 927,
 934,
 940,
 946,
 949,
 954,
 963,
 964,
 971,
 975,
 980,
 983,
 989,
 990,
 992,
 995,
 1059,
 1096,
 1110,
 1322,
 1366,
 1376,
 1379,
 1392,
 1401,
 1413,
 1481,
 1496,
 1526,
 1578,
 1605,
 1608,
 1611,
 1614,
 1617,
 1618,
 1621,
 1625,
 1643,
 1644,
 1658,
 1680,
 1698,
 1702,
 1707,
 1754,
 1755,
 1765,
 1775,
 1789,
 1814,
 1829,
 1830,
 1838,
 1916,
 1938,
 1961,
 2049,
 2120,
 2121,
 2123,
 2124,
 2125,
 2127,
 2132,
 2135,
 2139,
 2145,
 2152,
 2156,
 2158,
 2161,
 2162,
 2166,
 2172,
 2183,
 2184,
 2192,
 2194,
 2199,
 2206,
 2207,
 2209,
 2214,
 2217,
 2219,
 2224,
 2228,
 2229,
 2230,
 2237,
 2239,
 2243,
 2245,
 2256,
 2257,
 2262,
 2270,
 2279,
 2282,
 2283,
 2289,
 2293,
 2295,
 2308,
 2310,
 231